In [1]:
import transformers
import numpy as np
import pandas as pd
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')
import time

In [3]:
dfa = pd.read_csv('merge.csv')
dfa = dfa.drop(['split', 'name' ], axis = 1)
print(len(dfa))
print(dfa.head())

42409
   Unnamed: 0                                               text  label
0           0   F. NARIMAN, J. Leave granted. In 2008, the Pu...      1
1           1   S. THAKUR, J. Leave granted. These appeals ar...      0
2           2   Markandey Katju, J. Leave granted. Heard lear...      1
3           3   ALTAMAS KABIR,J. Leave granted. The question ...      1
4           4   CIVIL APPEAL NO. 598 OF 2007 K. MATHUR, J. Th...      1


In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
iterations = int(len(dfa)/10) + 1
c = -1
print(iterations)
for i in range(int(iterations)):
    t0 = time.time()
    print('Iteration no. :' + str(i+1))
    df = dfa[i*10:(i+1)*10]
    index = []
    label = []
    tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
    max_len = 510
    our_tokens = []
    for j in range(len(tokenized.values)): #sep-cls $ sep-cls
      c += 1
      cur_tokens = tokenized.values[j]
      cur_tokens = cur_tokens[::-1] # reversed
      for k in range(5):
        y = k
        if len(cur_tokens) < y*510:
          break
        index.append(c)
        label.append(df['label'][i*10+j])
        tokens_512 = cur_tokens[y*510:(y+1)*510]
        tokens_512 = tokens_512[::-1] # reversed back to original
        if y == 0:
          tokens_512.insert(-1, 102) #sep
        elif y == int(len(cur_tokens)//510):
          tokens_512.insert(0, 101) #cls
        else:
          tokens_512.insert(-1, 102) #both
          tokens_512.insert(0, 101)
        our_tokens.append(tokens_512)
    max_len = 512
    padded = np.array([i + [0]*(max_len-len(i)) for i in our_tokens])
    attention_mask = np.where(padded != 0, 1, 0)
    attention_mask.shape
    input_ids = torch.tensor(padded)  
    attention_mask = torch.tensor(attention_mask)
    with torch.no_grad():
        last_hidden_states = model(input_ids, attention_mask=attention_mask)
        features = last_hidden_states[0][:,0,:].numpy()

    features = pd.DataFrame(features)
    features['uid'] = index
    features['label'] = label
    features.to_csv('features_bb_entire/features__bb_' + str(i) + '_.csv',index=False)
    t1 = time.time()
    print(str((t1-t0)/60) + ' mins')

4241
Iteration no. :1
